In [10]:
#calculation
import pandas as pd
import numpy as np
from scipy import stats
import csv
import scipy.stats as stats

#plot
import matplotlib.pyplot as plt
import seaborn as sns

#MCMC package
import pymc3 as pm
import theano as T

#tokenizer packages
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

#Spelling Check
from textblob import Word
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jackritian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jackritian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jackritian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jackritian/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [41]:
training_tokens = np.load('data/training_tokens.npy')
attention_mask = np.load('data/attention_mask.npy')
data_df = pd.read_csv('data/train.csv')
train_text = data_df['full_text']

#dictionary
dictionary_df = pd.read_csv('data/new_dictionary.csv')
index = dictionary_df['Unnamed: 0'].to_list()
dictionary_df = dictionary_df.drop(columns = 'Unnamed: 0')
dictionary_df.index = index
dictionary_len = dictionary_df.shape[1]


In [67]:
frequency_list = dictionary_df.loc['sample_frequency']
dictionary_df.loc['bag_lable']=pd.qcut(frequency_list, q = np.linspace(0,1,11), duplicates = 'drop', labels = [1,2,3,4,5,6,7,8])

In [69]:
dictionary = pd.read_csv('data/dictionary_with_bagging.csv')
index = dictionary['Unnamed: 0'].to_list()
dictionary = dictionary.drop(columns = 'Unnamed: 0')
dictionary.index = index
dictionary_len = dictionary.shape[1]

In [86]:
dictionary_df

,think,student,would,benefit,learning,home,wont,change,get,early,...,irritate,mist,natalie,boxer,mike,slang,prettiest,classical,nd,dived
position_index,0.000000e+00,1.000000e+00,2.000000e+00,3.000000,4.000000,5.000000e+00,6.00000,7.000000e+00,8.000000e+00,9.000000,...,7559.000000,7560.000000,7561.000000,7562.000000,7563.0000,7564.000000,7565.000000,7566.000000,7567.000000,7568.000000
word_len,5.000000e+00,7.000000e+00,5.000000e+00,7.000000,8.000000,4.000000e+00,4.00000,6.000000e+00,3.000000e+00,5.000000,...,8.000000,4.000000,7.000000,5.000000,4.0000,5.000000,9.000000,9.000000,2.000000,5.000000
sample_frequency,6.382000e+03,1.713800e+04,7.154000e+03,1336.000000,1477.000000,3.058000e+03,341.00000,2.770000e+03,8.653000e+03,902.000000,...,1.000000,1.000000,4.000000,1.000000,3.0000,1.000000,1.000000,2.000000,1.000000,1.000000
importance,1.923036e-12,5.165146e-28,1.466908e-13,0.000039,0.000024,1.247298e-07,0.00107,3.257565e-07,9.916953e-16,0.000165,...,0.003322,0.003322,0.003289,0.003322,0.0033,0.003322,0.003322,0.003311,0.003322,0.003322
bag_lable,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000e+00,8.00000,8.000000e+00,8.000000e+00,8.000000,...,1.000000,1.000000,3.000000,1.000000,2.0000,1.000000,1.000000,1.000000,1.000000,1.000000


In [49]:
last_four = training_tokens[:,-4:]

In [83]:
exposure = training_tokens.shape

(3911, 7573)

In [84]:
bagging_token = np.zeros((attention_mask.shape[0],8))
for i in range(attention_mask.shape[0]):
    for j in range(training_tokens.shape[1]-4):
        word = dictionary_df.columns[j]
        bag_num = dictionary_df[word]['bag_lable']
        bagging_token[i,int(bag_num)-1] += training_tokens[i,j]

In [91]:
bagging_token.shape

(3911, 8)

In [94]:
bagged_training_token = np.column_stack((bagging_token,last_four))

In [96]:
np.save('data/bagged_training_token.npy', bagged_training_token)